# Imports:

In [1]:
import mne
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, GridSearchCV
from mne.decoding import CSP
from sklearn.metrics import classification_report, confusion_matrix
import copy
import sys, os
import pandas as pd
import time
from scipy import signal
import scipy
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import torch

In [2]:
!pwd
# projects/def-b09sdp/bijan/Phase2/P16.fdt

'pwd' is not recognized as an internal or external command,
operable program or batch file.


# Functions:

Mode: 'binary', '3-class', '4-class', '5-class', '6-class'

In [3]:
def data_wrapper(data_epochs, data_labels, mode='binary'):
    
    if mode == 'binary':
        epochs = []
        labels = []

        for i in range(len(data_epochs)):
            participant_epochs = data_epochs[i]
            participant_labels = data_labels[i]
    
            binary_epochs = participant_epochs[(participant_labels==1) | (participant_labels==2)]
            #class2_epochs = participant_epochs[participant_labels==2]
            #bi_epochs = np.concatenate((class1_epochs, class2_epochs), axis=0)
            epochs.append(binary_epochs)
    
            binary_labels = participant_labels[(participant_labels==1) | (participant_labels==2)]
            #bi_labels = np.concatenate((class1_labels, class2_labels), axis=0)
            labels.append(binary_labels)
            
    elif mode == '3_class':
        
        epochs = []
        labels = []
        
        for i in range(len(data_epochs)):
            participant_epochs = data_epochs[i]
            participant_labels = data_labels[i]
    
            multiclass_epochs = participant_epochs[(participant_labels==1) | (participant_labels==2) | (participant_labels==3)]
            epochs.append(multiclass_epochs)
    
            multiclass_labels = participant_labels[(participant_labels==1) | (participant_labels==2) | (participant_labels==3)]
            labels.append(multiclass_labels)
            
            
    elif mode == '4_class_RS':
        
        epochs = []
        labels = []
        
        for i in range(len(data_epochs)):
            participant_epochs = data_epochs[i]
            participant_labels = data_labels[i]
    
            multiclass_epochs = participant_epochs[(participant_labels==2) | (participant_labels==6) | (participant_labels==5) | (participant_labels==1)]
            epochs.append(multiclass_epochs)
    
            multiclass_labels = participant_labels[(participant_labels==2) | (participant_labels==6) | (participant_labels==5) | (participant_labels==1)]
            labels.append(multiclass_labels)
        
    elif mode == '4_class_LS':
        
        epochs = []
        labels = []
        
        for i in range(len(data_epochs)):
            participant_epochs = data_epochs[i]
            participant_labels = data_labels[i]
    
            multiclass_epochs = participant_epochs[(participant_labels==3) | (participant_labels==7) | (participant_labels==5) | (participant_labels==1)]
            epochs.append(multiclass_epochs)
    
            multiclass_labels = participant_labels[(participant_labels==3) | (participant_labels==7) | (participant_labels==5) | (participant_labels==1)]
            labels.append(multiclass_labels)
    elif mode == '6_class':
        
        epochs = []
        labels = []
        
        for i in range(len(data_epochs)):
            participant_epochs = data_epochs[i]
            participant_labels = data_labels[i]
    
            multiclass_epochs = participant_epochs[(participant_labels==3) | (participant_labels==7) | (participant_labels==5) | (participant_labels==6) | (participant_labels==2) | (participant_labels==1)]
            epochs.append(multiclass_epochs)
    
            multiclass_labels = participant_labels[(participant_labels==3) | (participant_labels==7) | (participant_labels==5) | (participant_labels==6) | (participant_labels==2) | (participant_labels==1)]
            labels.append(multiclass_labels)
    
    return epochs, labels

In [20]:
def encoder(y_data, method=OneHotEncoder):
    
    encoder = OneHotEncoder()
    encoder.fit(y_data[0].reshape(-1, 1))
    
    for i in range(len(y_data)):
        
        a = encoder.transform(y_data[i].reshape(-1, 1))
        y_data[i] = a.toarray()

    return y_data

In [5]:
def calc_csp(x_train, y_train, x_test):
    
        csp = CSP(number_of_components)
        csp_fit = csp.fit(x_train, y_train)
        train_feat = csp_fit.transform(x_train)
        test_feat = csp_fit.transform(x_test)
        return train_feat, test_feat
    

In [6]:
def feature_extraction(dataset, labels, number_of_bands, test_data):

    low_cutoff = 0
    
    for b in range(number_of_bands):
        low_cutoff += 4
        data = dataset.copy()
        data_test = test_data.copy()
        filtered_data = mne.filter.filter_data(data, sampling_freq, low_cutoff, low_cutoff + 4, verbose = False)
        filtered_data_test = mne.filter.filter_data(test_data, sampling_freq, low_cutoff, low_cutoff + 4, verbose = False)
        [train_feats, test_feats] = calc_csp(filtered_data, labels[:,0], filtered_data_test)
        if b == 0:
            train_features = train_feats
            test_features = test_feats
        else:
            train_features = np.concatenate((train_features, train_feats), axis = 1)
            test_features = np.concatenate((test_features, test_feats), axis = 1)
    
    return train_features, test_features

In [7]:
def feature_selection(train_features, labels, number_of_selected_features):
    X = pd.DataFrame(train_features)
    y = pd.DataFrame(labels)
    K = number_of_selected_features
    
    df = pd.concat([y,X], axis = 1)
    df.columns = df.columns.astype(str)
        
    selected_features = list(map(int, pymrmr.mRMR(df, 'MID', K)))
    return selected_features

# Device (GPU\CPU\MPS)

In [8]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


# Data reading:

\
\
Read all the data except participant 1 (P2 - P15)

In [9]:
if device == 'cuda':
    print("Running on cloud ...")
    print("Please make sure to modify how you read the data according to your need!\n\n")
    raw_data_path = "/home/bijan/projects/def-b09sdp/bijan/Phase2/"
    data_epochs = []
    data_labels = []

    for participant_id in range(1, 10):
        participant = f"P{participant_id}"
        file_path = f"{raw_data_path}/{participant}.set"
        epochs = mne.io.read_epochs_eeglab(file_path)
        data_epochs.append(epochs.get_data())
        data_labels.append(epochs.events[:, -1])

elif device == 'cpu':
    print("Running local ...")
    print("Please make sure to change the data path!\n\n")
    raw_data_path = "D:/Research Dr. Power/BCI_IV_2a/BCICIV_2a_gdf"
    train_data_epochs = []
    train_data_labels = []
    test_data_epochs = []
    test_data_labels = []
    for participant_id in range(1, 10):
        participant_E = f"A0{participant_id}E"
        participant_T = f"A0{participant_id}T"
        
        file_path_E = f"{raw_data_path}/{participant_E}.gdf"
        file_path_T = f"{raw_data_path}/{participant_T}.gdf"
        
        raw_train = mne.io.read_raw_gdf(file_path_T, eog=['EOG-left', 'EOG-central', 'EOG-right'], preload=True).drop_channels(['EOG-left', 'EOG-central', 'EOG-right'])
        raw_test = mne.io.read_raw_gdf(file_path_E, eog=['EOG-left', 'EOG-central', 'EOG-right'], preload=True).drop_channels(['EOG-left', 'EOG-central', 'EOG-right'])
        
        raw_train.set_eeg_reference()
        raw_test.set_eeg_reference()
        
        train_events = mne.events_from_annotations(raw_train)
        test_events = mne.events_from_annotations(raw_test)
        
        tmin = 0.504
        tmax = 2.5
        
        try:
            train_epochs = mne.Epochs(raw_train, train_events[0], event_id=[7,8,9,10], on_missing ='warn', tmin=tmin, tmax=tmax, baseline=None)
            test_epochs = mne.Epochs(raw_test, train_events[0], event_id=[7,8,9,10], on_missing ='warn', tmin=tmin, tmax=tmax, baseline=None)
        except ValueError:
            train_epochs = mne.Epochs(raw_train, train_events[0], event_id=[7,8,9,10], on_missing ='warn', tmin=tmin, tmax=tmax, baseline=(0, 0))
            test_epochs = mne.Epochs(raw_test, train_events[0], event_id=[7,8,9,10], on_missing ='warn', tmin=tmin, tmax=tmax, baseline=(0, 0))
        train_data_epochs.append(train_epochs.get_data())
        test_data_epochs.append(test_epochs.get_data())
        
        train_data_labels.append(train_epochs.events[:, -1])
        test_data_labels.append(test_epochs.events[:, -1])

Running local ...
Please make sure to change the data path!


Extracting EDF parameters from D:\Research Dr. Power\BCI_IV_2a\BCICIV_2a_gdf\A01T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG
Creating raw.info structure...
Reading 0 ... 672527  =      0.000 ...  2690.108 secs...


C:\ProgramData\Anaconda3\envs\BCI_2\lib\contextlib.py:126: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


Extracting EDF parameters from D:\Research Dr. Power\BCI_IV_2a\BCICIV_2a_gdf\A01E.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG
Creating raw.info structure...
Reading 0 ... 686999  =      0.000 ...  2747.996 secs...


C:\ProgramData\Anaconda3\envs\BCI_2\lib\contextlib.py:126: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']
Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '783']
Not setting metadata
288 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
288 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 288 events and 500 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 288 events and 500 original time points ...
0 bad epochs dropped
Extracting EDF parameters from D:\Research Dr. Power\BCI_IV_2a\BCICIV_2a_gdf\A02T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type o

...

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']
Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '783']
Not setting metadata
288 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
288 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 288 events and 500 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 288 events and 500 original time points ...
0 bad epochs dropped
Extracting EDF parameters from D:\Research Dr. Power\BCI_IV_2a\BCICIV_2a_gdf\A09T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type o

C:\ProgramData\Anaconda3\envs\BCI_2\lib\contextlib.py:126: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


Extracting EDF parameters from D:\Research Dr. Power\BCI_IV_2a\BCICIV_2a_gdf\A09E.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG
Creating raw.info structure...
Reading 0 ... 675097  =      0.000 ...  2700.388 secs...


C:\ProgramData\Anaconda3\envs\BCI_2\lib\contextlib.py:126: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']
Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '783']
Not setting metadata
288 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
288 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 288 events and 500 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 288 events and 500 original time points ...
0 bad epochs dropped


In [13]:
train_events[0]

array([[     0,      0,      5],
       [     0,      0,      3],
       [ 30878,      0,      5],
       ...,
       [669791,      0,      8],
       [671350,      0,      6],
       [671850,      0,      9]])

In [14]:
train_events[1]

{'1023': 1,
 '1072': 2,
 '276': 3,
 '277': 4,
 '32766': 5,
 '768': 6,
 '769': 7,
 '770': 8,
 '771': 9,
 '772': 10}

In [15]:
raw_train[0][1].shape, raw_train[0][0].shape

((673328,), (1, 673328))

In [16]:
for i in range(9):
    print("Subject {} train epochs:    ".format(i+1), train_data_epochs[i].shape, "    Subject {} train labels:    ".format(i+1), train_data_labels[i].shape)
    print("Subject {} test epochs:     ".format(i+1), test_data_epochs[i].shape, "    Subject {} test labels:     ".format(i+1), test_data_labels[i].shape)

Subject 1 train epochs:     (288, 22, 500)     Subject 1 train labels:     (288,)
Subject 1 test epochs:      (288, 22, 500)     Subject 1 test labels:      (288,)
Subject 2 train epochs:     (288, 22, 500)     Subject 2 train labels:     (288,)
Subject 2 test epochs:      (281, 22, 500)     Subject 2 test labels:      (281,)
Subject 3 train epochs:     (288, 22, 500)     Subject 3 train labels:     (288,)
Subject 3 test epochs:      (282, 22, 500)     Subject 3 test labels:      (282,)
Subject 4 train epochs:     (144, 22, 500)     Subject 4 train labels:     (144,)
Subject 4 test epochs:      (144, 22, 500)     Subject 4 test labels:      (144,)
Subject 5 train epochs:     (288, 22, 500)     Subject 5 train labels:     (288,)
Subject 5 test epochs:      (285, 22, 500)     Subject 5 test labels:      (285,)
Subject 6 train epochs:     (288, 22, 500)     Subject 6 train labels:     (288,)
Subject 6 test epochs:      (282, 22, 500)     Subject 6 test labels:      (282,)
Subject 7 train 

In [17]:
all_data_epochs = np.zeros(len(train_data_epochs)).tolist()
all_data_labels = np.zeros(len(train_data_epochs)).tolist()

for i in range(len(train_data_epochs)):
    all_data_epochs[i] = np.concatenate((train_data_epochs[i], test_data_epochs[i]), axis=0)
    all_data_labels[i] = np.concatenate((train_data_labels[i], test_data_labels[i]), axis=0)

In [18]:
# Checking if the concatenation does not have a problem!

print("Specific element in train and test sets:      ", train_data_epochs[i][10, 10, 10], test_data_epochs[i][100, 13, 14])
print("The same element in all data set combined:    ", all_data_epochs[i][10, 10, 10], all_data_epochs[i][388, 13, 14])
print("\n\n")
print("Checking the labels in train and test:        ", train_data_labels[i][10], test_data_labels[i][100])
print("The same element in all data set combined:    ", all_data_labels[i][10], all_data_labels[i][388])

Specific element in train and test sets:       -1.5491832386363597e-06 6.613991477272725e-06
The same element in all data set combined:     -1.5491832386363597e-06 6.613991477272725e-06



Checking the labels in train and test:         7 8
The same element in all data set combined:     7 8


In [19]:
for i in range(len(all_data_epochs)):
    print("All data epoch shape (sub {}):    ".format(i), all_data_epochs[i].shape, "    All label shape (sub {}):     ".format(i), all_data_labels[i].shape)

All data epoch shape (sub 0):     (576, 22, 500)     All label shape (sub 0):      (576,)
All data epoch shape (sub 1):     (569, 22, 500)     All label shape (sub 1):      (569,)
All data epoch shape (sub 2):     (570, 22, 500)     All label shape (sub 2):      (570,)
All data epoch shape (sub 3):     (288, 22, 500)     All label shape (sub 3):      (288,)
All data epoch shape (sub 4):     (573, 22, 500)     All label shape (sub 4):      (573,)
All data epoch shape (sub 5):     (570, 22, 500)     All label shape (sub 5):      (570,)
All data epoch shape (sub 6):     (572, 22, 500)     All label shape (sub 6):      (572,)
All data epoch shape (sub 7):     (576, 22, 500)     All label shape (sub 7):      (576,)
All data epoch shape (sub 8):     (576, 22, 500)     All label shape (sub 8):      (576,)


# Encoding

In [21]:
no_encode = copy.deepcopy(all_data_labels)
encoded = encoder(all_data_labels)


In [23]:
print("Epochs Length:", len(all_data_labels))
print("labels Length:", len(encoded))
print('\n\n')
print("Participant 16 - Epochs[0] shape:", no_encode[3].shape)
print("Participant 16 - labels[0] shape:", encoded[3].shape)
print('\n\n')
print("Participant 16 - labels[0]:")
print(all_data_labels[3])

Epochs Length: 9
labels Length: 9



Participant 16 - Epochs[0] shape: (288,)
Participant 16 - labels[0] shape: (288, 4)



Participant 16 - labels[0]:
[[0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 ...
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]]


# Checking the criterion:

# Within Subject:

In [29]:
participants = [1, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15]
all_tests_true = []
all_tests_pred = []
max_epochs = 500
learning_rate=0.0001
weight_decay=0.25*learning_rate

confusion_matrices = []


for i in range(len(EEG_epochs)):

    epoch_participant = EEG_epochs[i]
    encoded_participant = encoded[i]
    
    split_index = 40 # The last n = split_index samples are in the test set
    
    train_epoch, test_epoch = epoch_participant[:-1*split_index], epoch_participant[-1*split_index:]
    train_label, test_labels = encoded_participant[:-1*split_index], encoded_participant[-1*split_index:]
    no_encode_train_label = no_encode[i][:-1*split_index]
    no_encode_test_label = no_encode[i][-1*split_index:]
    
    # print(train_epoch.shape)
    # print(train_label.shape)
    # print(len(no_encode_train_label))
    
    batch_size = 12  # Set your batch size
    fold_num = int(EEG_epochs[i].shape[0] // batch_size)
    

    
    #model = Deep4Net(in_chans=64, n_classes=4, input_window_samples=1123, final_conv_length='auto')
    
    model = new_model
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    
    model.to(device)
    model.train()
    
    patience = 300
    best_metric = float('inf')
    counter = 1
    best_model_state = model.state_dict()
    
    train_loss_epochs = []
    train_acc_epochs = []
    test_loss_epochs = []
    test_acc_epochs = []
    
    
    
    models = []
    
    
    for epoch in range(max_epochs):
        
        temp_train_pred = []
        temp_train_true = []
        
        train_running_loss = 0
        
        stratified_kfold = StratifiedKFold(n_splits=fold_num, random_state=1, shuffle=True)
        
        for _, (_, train_batch_index) in enumerate(stratified_kfold.split(train_epoch, no_encode_train_label)):
            
            batch_epoch, batch_label = train_epoch[train_batch_index], train_label[train_batch_index]
            
            train_dataset = EEG_Dataset(batch_epoch, batch_label, transform=None)
            train_loader = DataLoader(train_dataset, batch_size=batch_epoch.shape[0])
        
        
            for train_inputs, train_labels in train_loader:
                #print(train_labels)
                train_inputs, train_labels = train_inputs.to(torch.float32).to(device), train_labels.to(torch.float32).to(device)
            
                optimizer.zero_grad()
                train_outputs = model(train_inputs)
                loss = criterion(train_outputs, train_labels)
                loss.backward()
                optimizer.step()
                train_running_loss += loss.item()
                y_pred_train = torch.argmax(train_outputs, 1).tolist()
                y_true_train = torch.argmax(train_labels, 1).tolist()
                temp_train_pred.extend(y_pred_train)
                temp_train_true.extend(y_true_train)
                
        test_dataset = EEG_Dataset(test_epoch, test_labels, transform=None)
        test_loader = DataLoader(test_dataset, batch_size=test_epoch.shape[0])
            
        with torch.no_grad():
            for test_input, test_label in test_loader:
                
                test_input, test_label = test_input.to(torch.float32).to(device), test_label.to(torch.float32).to(device)
                test_output = model(test_input)
                test_loss = criterion(test_output, test_label).item()
                y_pred_test = torch.argmax(test_output, 1).tolist()
                y_true_test = torch.argmax(test_label, 1).tolist()
                test_conf_mat = confusion_matrix(y_true_test, y_pred_test)
                test_accuracy = np.trace(test_conf_mat) / test_conf_mat.sum()
        
        avg_train_loss = train_running_loss / fold_num
        
        train_loss_epochs.append(avg_train_loss)
        test_loss_epochs.append(test_loss)
        test_acc_epochs.append(test_accuracy)
        train_conf_mat = confusion_matrix(temp_train_true, temp_train_pred)
        train_accuracy = np.trace(train_conf_mat) / train_conf_mat.sum()
        train_acc_epochs.append(train_accuracy)
        
        if epoch % 5 == 0:
            print(f"Epoch [{(epoch + 1): 3d}/{max_epochs}] - Avg Training Loss: {avg_train_loss:.4f} Train Accuracy: {train_accuracy:.2f} - Test Loss: {test_loss:.2f} - Test Accuracy: {test_accuracy:.2f}")
        
        if test_loss < best_metric:
            best_metric = test_loss
            best_model_state = model.state_dict()
            counter = 1
        
        else:
            counter += 1
        
        if counter > patience:
            break
        best_epoch = epoch-patience+2
        epochs_range = np.arange(1, len(train_loss_epochs[:best_epoch])+1)
        
        
    model.load_state_dict(best_model_state)
    
    
                
    test_input, test_labels = torch.tensor(test_epoch, dtype=torch.float32).to(device), torch.tensor(test_labels, dtype=torch.float32).to(device)
    test_output = model(test_input)
    y_pred_test = torch.argmax(test_output, 1).tolist()
    y_true_test = torch.argmax(test_labels, 1).tolist()
    all_tests_true.append(y_true_test)
    all_tests_pred.append(y_pred_test)
        
    
    
    models.append(best_model_state)
    
    
    
    plt.figure(figsize=(12, 6))
    plt.suptitle("Participant {}".format(participants[i]))

    plt.subplot(1, 2, 1)
    plt.title("Loss")
    plt.plot(epochs_range, train_loss_epochs[:best_epoch])
    plt.plot(epochs_range, test_loss_epochs[:best_epoch])
    plt.legend(["Train", "Test"])

    plt.subplot(1, 2, 2)
    plt.title("Accuracy")
    plt.plot(epochs_range, train_acc_epochs[:best_epoch])
    plt.plot(epochs_range, test_acc_epochs[:best_epoch])
    plt.legend(["Train", "Test"])
    plt.savefig("P{}_within_subject.jpg".format(participants[i]))
    plt.show()
    plt.clf()

Epoch [  1/500] - Avg Training Loss: 1.7928 Train Accuracy: 0.16 - Test Loss: 1.80 - Test Accuracy: 0.28
Epoch [  6/500] - Avg Training Loss: 1.7879 Train Accuracy: 0.17 - Test Loss: 1.84 - Test Accuracy: 0.00
Epoch [ 11/500] - Avg Training Loss: 1.7936 Train Accuracy: 0.18 - Test Loss: 1.85 - Test Accuracy: 0.00
Epoch [ 16/500] - Avg Training Loss: 1.7871 Train Accuracy: 0.18 - Test Loss: 1.86 - Test Accuracy: 0.00
Epoch [ 21/500] - Avg Training Loss: 1.7876 Train Accuracy: 0.21 - Test Loss: 1.84 - Test Accuracy: 0.00
Epoch [ 26/500] - Avg Training Loss: 1.7900 Train Accuracy: 0.18 - Test Loss: 1.85 - Test Accuracy: 0.00


KeyboardInterrupt: 

In [29]:
print(len(all_tests_pred))

0


In [99]:
print(all_tests_pred[3])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


In [100]:
for i in range(len(all_tests_true)):
   print(all_tests_true[i])


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1

In [101]:
confusion_matrices = []
for i in range(len(all_tests_pred)):
    y_true = all_tests_true[i]
    y_pred = all_tests_pred[i]
    confusion_matrices.append(confusion_matrix(y_true, y_pred))

In [102]:
print(confusion_matrices[1])

[[5 1 3 1]
 [0 5 0 5]
 [2 3 5 0]
 [0 4 3 3]]


In [103]:
summation = pd.DataFrame(sum(confusion_matrices), index=['class 1 (True)', 'class 2 (True)', 'class 3 (True)', 'class 4 (True)'], columns=['class 1 (Pred)', 'class 2 (Pred)', 'class 3 (Pred)', 'class 4 (Pred)'])
summation

,class 1 (Pred),class 2 (Pred),class 3 (Pred),class 4 (Pred)
class 1 (True),73,18,28,11
class 2 (True),12,59,23,36
class 3 (True),27,19,68,16
class 4 (True),12,45,24,49


In [104]:
summation * np.eye(4, 4) / 130

,class 1 (Pred),class 2 (Pred),class 3 (Pred),class 4 (Pred)
class 1 (True),0.561538,0.000000,0.000000,0.000000
class 2 (True),0.000000,0.453846,0.000000,0.000000
class 3 (True),0.000000,0.000000,0.523077,0.000000
class 4 (True),0.000000,0.000000,0.000000,0.376923


In [105]:
np.sum(summation * np.eye(4, 4) / 130).sum() / 4

/home/bijan/py3x/lib/python3.10/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


0.4788461538461538

In [ ]:
mean_confusion_matrix = summation / len(confusion_matrices)
mean_confusion_matrix

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Assuming 'mean_confusion_matrix' is the mean confusion matrix NumPy array
mean_confusion_matrix = sum(confusion_matrices) / len(confusion_matrices)

true_positive = mean_confusion_matrix[1, 1]
true_negative = mean_confusion_matrix[0, 0]
false_positive = mean_confusion_matrix[0, 1]
false_negative = mean_confusion_matrix[1, 0]

accuracy = (true_positive + true_negative) / (true_positive + true_negative + false_positive + false_negative)
precision = true_positive / (true_positive + false_positive)
recall = true_positive / (true_positive + false_negative)
sensitivity = recall  # Same as recall
f1 = 2 * (precision * recall) / (precision + recall)

print("Average metrics for binary classification (Left vs Right hand):\n\n")
print(f"       Accuracy:                  {accuracy:.2f}\n")
print(f"       Precision:                 {precision:.2f}\n")
print(f"       Recall (Sensitivity):      {recall:.2f}     \n")
print(f"       F1 Score:                  {f1:.2f}\n")


# Cross-subjects (WO hyperparameter tuning):

In [ ]:
participants = [1, 2, 3, 4, 5, 6, 7, 8, 9]
class_numbers=4
num_subjects = len(all_data_epochs)
all_tests_true = []
all_tests_pred = []
max_epochs = 500
learning_rate=0.0009



kf_outer2 = KFold(n_splits=num_subjects, shuffle=True, random_state=2)    # Split the data into Train_CrossVal and test sets.



for i, (train_index, test_index) in enumerate(kf_outer2.split(all_data_epochs)):
    
    
    
    train_epochs = np.concatenate([all_data_epochs[j] for j in train_index])
    test_epochs = np.concatenate([all_data_epochs[k] for k in test_index])
    train_labels = np.concatenate([encoded[l] for l in train_index])
    test_labels = np.concatenate([encoded[m] for m in test_index])
    no_encoded_train_labels = np.concatenate([no_encode[n] for n in train_index])
    no_encoded_test_labels = np.concatenate([no_encode[o] for o in test_index])
    train_ids_for_save = [participants[i] for i in train_index]
    test_ids_for_save = [participants[i] for i in test_index]
    
    
    print("Outer Loop {}".format(i+1), "\n")
    print("      Train epochs' shape:                               ", train_epochs.shape)

    print("      Test epochs' shape:                                ", test_epochs.shape)
    print("      Test labels' shape:                                ", test_labels.shape)
    print("      Train labels' shape (without encoding):            ", no_encoded_train_labels.shape)

    print("      Test labels' shape (without encoding):             ", no_encoded_test_labels.shape)
    print("      Train index:                                       ", train_ids_for_save)

    print("      Test index:                                        ", test_ids_for_save)
    print('\n\n')
    
    
    # Create the EEGNet model
    model = EEGNet(nb_classes=4, Chans=22, Samples=500, dropoutRate=0.5, kernLength=22, F1=8, D=2, F2=16, norm_rate=0.25, dropoutType='Dropout')

    # Compile the model
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    patience = 60
    train_loss_epochs = []
    train_acc_epochs = []
    test_loss_epochs = []
    test_acc_epochs = []
    train_conf_mat = []
    
    # Define early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)

    
    # Normalizing the features
    mean = train_epochs.mean(axis=(0, 2, 3), keepdims=True)
    std = train_epochs.std(axis=(0, 2, 3), keepdims=True)
    
    print(mean.shape)
    print(std.shape)
    
    norm_train_epochs = (train_epochs - mean) / std
    norm_test_epochs = (test_epochs - mean) / std
    
    
    
    
    # Train the model
    history = model.fit(
        norm_train_epochs, train_labels,
        validation_data=(norm_test_epochs, test_labels),
        epochs=max_epochs,
        batch_size=32,  # Adjust as needed
        callbacks=[early_stopping]
    )

    # Collect training metrics
    train_loss_epochs.extend(history.history['loss'])
    train_acc_epochs.extend(history.history['accuracy'])

    # Collect testing metrics
    test_loss_epochs.extend(history.history['val_loss'])
    test_acc_epochs.extend(history.history['val_accuracy'])
    
    epochs_range = np.arange(1, len(train_loss_epochs[:-1*patience])+1)
    
    plt.figure(figsize=(12, 6))
    plt.suptitle("Participant {}".format(participants[test_index[0]]))

    plt.subplot(1, 2, 1)
    plt.title("Loss")
    plt.plot(epochs_range, train_loss_epochs[:-1*patience])
    plt.plot(epochs_range, test_loss_epochs[:-1*patience])
    plt.legend(["Train", "Test"])

    plt.subplot(1, 2, 2)
    plt.title("Accuracy")
    plt.plot(epochs_range, train_acc_epochs[:-1*patience])
    plt.plot(epochs_range, test_acc_epochs[:-1*patience])
    plt.legend(["Train", "Test"])
    plt.savefig("P{}.jpg".format(participants[test_index[0]]))
    plt.show()
    plt.clf()

Outer Loop 1 

      Train epochs' shape:                                (4320, 62, 1000)
      Test epochs' shape:                                 (360, 62, 1000)
      Train index:                                       [1, 3, 4, 5, 7, 8, 10, 11, 12, 13, 14, 15]
      Test index:                                        [6]



Epoch [  1/500] - Avg Training Loss: 1.7934 Train Accuracy: 0.16 - Test Loss: 1.80 - Test Accuracy: 0.13
Epoch [  6/500] - Avg Training Loss: 1.7929 Train Accuracy: 0.16 - Test Loss: 1.79 - Test Accuracy: 0.19
Epoch [ 11/500] - Avg Training Loss: 1.7929 Train Accuracy: 0.16 - Test Loss: 1.79 - Test Accuracy: 0.19
Epoch [ 16/500] - Avg Training Loss: 1.7923 Train Accuracy: 0.17 - Test Loss: 1.79 - Test Accuracy: 0.14
Epoch [ 21/500] - Avg Training Loss: 1.7924 Train Accuracy: 0.17 - Test Loss: 1.79 - Test Accuracy: 0.15
Epoch [ 26/500] - Avg Training Loss: 1.7925 Train Accuracy: 0.17 - Test Loss: 1.80 - Test Accuracy: 0.15
Epoch [ 31/500] - Avg Training Loss: 1.792

In [32]:
len(all_tests_pred[2])

240

In [16]:
confusion_matrices = []
for i in range(len(all_tests_pred)):
    y_true = all_tests_true[i]
    y_pred = all_tests_pred[i]
    confusion_matrices.append(confusion_matrix(y_true, y_pred))

In [17]:
summation = pd.DataFrame(sum(confusion_matrices), index=['L (True)', 'LS (True)', 'S (True)', 'RS (True)', 'R (True)', 'Rest (True)'], columns=['L (Pred)', 'LS (Pred)', 'S (Pred)', 'RS (True)', 'R (True)', 'Rest (Pred)'])
summation

,L (Pred),LS (Pred),S (Pred),RS (True),R (True),Rest (Pred)
L (True),284,74,96,177,69,80
LS (True),75,285,79,75,214,52
S (True),82,90,257,69,84,198
RS (True),261,81,75,214,80,69
R (True),87,261,81,98,199,54
Rest (True),95,83,234,79,84,205


In [18]:
summation.sum(axis=1)

L (True)       780
LS (True)      780
S (True)       780
RS (True)      780
R (True)       780
Rest (True)    780
dtype: int64

In [19]:
summation / 780 * 100

,L (Pred),LS (Pred),S (Pred),RS (True),R (True),Rest (Pred)
L (True),36.410256,9.487179,12.307692,22.692308,8.846154,10.256410
LS (True),9.615385,36.538462,10.128205,9.615385,27.435897,6.666667
S (True),10.512821,11.538462,32.948718,8.846154,10.769231,25.384615
RS (True),33.461538,10.384615,9.615385,27.435897,10.256410,8.846154
R (True),11.153846,33.461538,10.384615,12.564103,25.512821,6.923077
Rest (True),12.179487,10.641026,30.000000,10.128205,10.769231,26.282051


In [ ]:
import pickle
with open("3class_all_tests_pred", "wb") as fp:
    pickle.dump(all_tests_pred, fp)

with open("3class_all_tests_true", "wb") as fp:
    pickle.dump(all_tests_true, fp)

    
print(all_tests_pred[1][1].shape)
print(all_tests_pred[12][1].shape)

In [ ]:
with open("3class_all_tests_pred", "rb") as fp:
    rand_var = pickle.load(fp)

    
with open("3class_all_tests_true", "rb") as fp:
    rand_var2 = pickle.load(fp)
print(rand_var[12][1].shape)
print(rand_var2[12][1].shape)

In [ ]:
all_tests_pred[1][2].shape

In [ ]:
np.argmax(all_tests_true[12][1], axis=1)+1

In [ ]:
a = np.array([[1, 2, 3], [4, 50, 6]])
np.sum(a, axis=1)

In [ ]:
y_true

In [ ]:
confusion_matrices_ap = []
y_pred_prob = []
for i in range(len(all_tests_pred)):
    y_pred = np.zeros(all_tests_true[i][0].shape)
    for j in range(len(all_tests_pred[1])):
        y_pred += all_tests_pred[i][j]
    y_pred_prob.append(y_pred/6)
    confusion_matrices_ap.append(confusion_matrix(np.argmax(all_tests_true[i][0], axis=1)+1, np.argmax(y_pred_prob[i], axis=1)+1))

In [ ]:
summation = pd.DataFrame(sum(confusion_matrices_ap), index=['class 1 (True)', 'class 2 (True)', 'class 3 (True)'], columns=['class 1 (Pred)', 'class 2 (Pred)', 'class 3 (Pred)'])
summation

In [ ]:
total_samples = summation.values.sum()
correct_predictions = summation.values.trace()
overall_accuracy = correct_predictions / total_samples

# Calculate accuracy for each class
class_accuracy = summation.values.diagonal() / summation.sum(axis=1)

# Print results
print(f"Overall Accuracy: {overall_accuracy:.2%}")

for i, acc in enumerate(class_accuracy):
    print(f"Accuracy for Class {i + 1}: {acc:.2%}")

In [ ]:
mean_confusion_matrix = sum(confusion_matrices_ap) / len(confusion_matrices_ap)

true_positive = mean_confusion_matrix[1, 1]
true_negative = mean_confusion_matrix[0, 0]
false_positive = mean_confusion_matrix[0, 1]
false_negative = mean_confusion_matrix[1, 0]

accuracy = (true_positive + true_negative) / (true_positive + true_negative + false_positive + false_negative)
precision = true_positive / (true_positive + false_positive)
recall = true_positive / (true_positive + false_negative)
sensitivity = recall  # Same as recall
f1 = 2 * (precision * recall) / (precision + recall)

print("Average metrics for binary classification (Left vs Right hand):\n\n")
print(f"       Accuracy:                  {accuracy:.2f}\n")
print(f"       Precision:                 {precision:.2f}\n")
print(f"       Recall (Sensitivity):      {recall:.2f}     \n")
print(f"       F1 Score:                  {f1:.2f}\n")


In [ ]:
summation / (13 * 40) * 100

In [ ]:
np.argmax([1, 1, 2, 2, 3, 3, 3, 1])

# Test for the effect of calibration:

In [32]:
models = []
Calibrated_model = []
for file in os.listdir("/home/bijan/py3x"):
    if file.endswith(".h5") and file.startswith("Calibrated"):
        Calibrated_model.append(file)
    elif file.endswith(".h5") and file.startswith("Model"):
        models.append(file)
        
Calibrated_model = sorted(Calibrated_model)
models = sorted(models)

In [49]:
models = {}
Calibrated_models = {}
for i in range(14):
    if i == 8:
        continue
        
    for j in range(6):
        #print("Model{}{}.h5".format(i+1, j+1))
        model_name = "Model{}{}.h5".format(i+1, j+1)
        Calibrated_model_name = "Calibrated_Model{}{}.h5".format(i+1, j+1)
        models["{:02}{:02}".format(i+1, j+1)] = load_model(model_name)
        Calibrated_models["{:02}{:02}".format(i+1, j+1)] = load_model(Calibrated_model_name)

In [50]:
"{:02}{:02}".format(2, 3)

'0203'

In [51]:
models["1406"].summary()

Model: "sequential_77"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_251 (Dense)           (None, 36)                20772     
                                                                 
 dense_252 (Dense)           (None, 4)                 148       
                                                                 
 dense_253 (Dense)           (None, 2)                 10        
                                                                 
Total params: 20,930
Trainable params: 20,930
Non-trainable params: 0
_________________________________________________________________


In [52]:
Calibrated_models["1406"].summary()

Model: "sequential_77"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_251 (Dense)           (None, 36)                20772     
                                                                 
 dense_252 (Dense)           (None, 4)                 148       
                                                                 
 dense_253 (Dense)           (None, 2)                 10        
                                                                 
Total params: 20,930
Trainable params: 20,930
Non-trainable params: 0
_________________________________________________________________


In [ ]:
num_subjects = len(EEG_epochs)
all_tests_true = []
all_tests_pred = []
models = {}
Calibrated_models = {}

kf_outer1 = KFold(n_splits=6, shuffle=True, random_state=42)              # Split the data into Train and Cross-Validation sets
kf_outer2 = KFold(n_splits=num_subjects, shuffle=True, random_state=2)    # Split the data into Train_CrossVal and test sets.


for i, (train_crossval_index, test_index) in enumerate(kf_outer2.split(EEG_epochs)):
    
    if test_index == 7:
        continue
    
    train_crossval = [EEG_epochs[i] for i in train_crossval_index]
    test_epochs = np.concatenate([EEG_epochs[i] for i in test_index])
    train_crossval_labels = [encoded[i] for i in train_crossval_index]
    test_labels = np.concatenate([encoded[i] for i in test_index])
    no_encoded_train_crossval = [no_encode[i] for i in train_crossval_index]
    no_encoded_test = np.concatenate([no_encode[i] for i in test_index])

    temp_pred = []
    temp_true = []

    for j, (train_index, val_index) in enumerate(kf_outer1.split(train_crossval)):
        
        
        train_epochs = np.concatenate([train_crossval[i] for i in train_index])
        crossval_epochs = np.concatenate([train_crossval[i] for i in val_index])
        train_labels = np.concatenate([train_crossval_labels[i] for i in train_index])
        crossval_labels = np.concatenate([train_crossval_labels[i] for i in val_index])
        no_encoded_train = np.concatenate([no_encoded_train_crossval[i] for i in train_index])
        no_encoded_crossval = np.concatenate([no_encoded_train_crossval[i] for i in val_index])
        train_ids_for_save = [train_crossval_index[i] for i in train_index]
        cross_val_ids_for_save = [train_crossval_index[i] for i in val_index]
        
        
        print("Outer Loop {} and Inner Loop {}:".format(i+1, j+1), "\n")
        print("      Train epochs' shape:                               ", train_epochs.shape)
        #print("     Train labels' shape:                               ", train_labels.shape)
        print("      Cross-validation epochs' shape:                    ", crossval_epochs.shape)
        #print("     Cross-validation labels' shape:                    ", crossval_labels.shape)
        print("      Test epochs' shape:                                ", test_epochs.shape)
        #print("     Test labels' shape:                                ", test_labels.shape)
        #print("     Train labels' shape (without encoding):            ", no_encoded_train.shape)
        #print("     Cross-validation labels' shape (without encoding): ", no_encoded_crossval.shape)
        #print("     Test labels' shape (without encoding):             ", no_encoded_test.shape)
        print("      Train index:                                      ", train_ids_for_save)
        print("      Cross-validation index:                           ", cross_val_ids_for_save)
        print("      Test index:                                       ", test_index)
        print('\n\n')
        
        original_stdout = sys.stdout
        sys.stdout = open('temp_stdout{}.txt'.format(i), 'w')  # Redirect output to a temporary file
        train_features, CrossVal_features, test_features = feature_extraction_cv(train_epochs, no_encoded_train, crossval_epochs, test_epochs, number_of_bands=9, sampling_freq=250, low_cutoff=0, number_of_components=64)
        sys.stdout.close()
        sys.stdout = original_stdout
    
        print("Train features shape:            ", train_features.shape)
        print("Cross-validation features shape: ", CrossVal_features.shape)
        print("Test features shape:             ", test_features.shape)
    
        model_name = "Model{}{}.h5".format(i+1, j+1)
        Calibrated_model_name = "Calibrated_Model{}{}.h5".format(i+1, j+1)
        models["{:02}{:02}".format(i+1, j+1)] = load_model(model_name)
        Calibrated_models["{:02}{:02}".format(i+1, j+1)] = load_model(Calibrated_model_name)
        
        temp_pred.append(models["{:02}{:02}".format(i+1, j+1)].predict(test_features[60:]))
        temp_true.append(test_labels[60:])
        
    all_tests_pred.append(temp_pred)
    all_tests_true.append(temp_true)
    


Outer Loop 1 and Inner Loop 1: 

      Train epochs' shape:                                (1337, 64, 1123)
      Cross-validation epochs' shape:                     (420, 64, 1123)
      Test epochs' shape:                                 (140, 64, 1123)
      Train index:                                       [1, 2, 3, 4, 5, 6, 7, 8, 10, 13]
      Cross-validation index:                            [0, 9, 12]
      Test index:                                        [11]



Train features shape:             (1337, 576)
Cross-validation features shape:  (420, 576)
Test features shape:              (140, 576)
Outer Loop 1 and Inner Loop 2: 

      Train epochs' shape:                                (1477, 64, 1123)
      Cross-validation epochs' shape:                     (280, 64, 1123)
      Test epochs' shape:                                 (140, 64, 1123)
      Train index:                                       [0, 1, 2, 3, 4, 6, 7, 9, 10, 12, 13]
      Cross-validation index:      

In [26]:
import pickle
with open("all_tests_pred_without_calibration", "wb") as fp:
    pickle.dump(all_tests_pred, fp)

with open("all_tests_true_without_calibration", "wb") as fp:
    pickle.dump(all_tests_true, fp)

In [27]:
confusion_matrices_ap = []
for i in range(len(all_tests_pred)):
    for j in range(len(all_tests_pred[1])):
        y_true = 2 - np.argmax(all_tests_true[i][j], axis=1)
        y_pred = 2 - np.argmax(all_tests_pred[i][j], axis=1)
    confusion_matrices_ap.append(confusion_matrix(y_true, y_pred))

In [28]:
summation = pd.DataFrame(sum(confusion_matrices_ap), index=['class 1 (True)', 'class 2 (True)'], columns=['class 1 (Pred)', 'class 2 (Pred)'])
summation

,class 1 (Pred),class 2 (Pred)
class 1 (True),358,162
class 2 (True),202,315


In [31]:
summation / 520

,class 1 (Pred),class 2 (Pred)
class 1 (True),0.688462,0.311538
class 2 (True),0.388462,0.605769


In [29]:
mean_confusion_matrix = sum(confusion_matrices_ap) / len(confusion_matrices_ap)

true_positive = mean_confusion_matrix[1, 1]
true_negative = mean_confusion_matrix[0, 0]
false_positive = mean_confusion_matrix[0, 1]
false_negative = mean_confusion_matrix[1, 0]
 
accuracy = (true_positive + true_negative) / (true_positive + true_negative + false_positive + false_negative)
precision = true_positive / (true_positive + false_positive)
recall = true_positive / (true_positive + false_negative)
sensitivity = recall  # Same as recall
f1 = 2 * (precision * recall) / (precision + recall)

print("Average metrics for binary classification (Left vs Right hand):\n\n")
print(f"       Accuracy:                  {accuracy:.2f}\n")
print(f"       Precision:                 {precision:.2f}\n")
print(f"       Recall (Sensitivity):      {recall:.2f}     \n")
print(f"       F1 Score:                  {f1:.2f}\n")


Average metrics for binary classification (Left vs Right hand):


       Accuracy:                  0.65

       Precision:                 0.66

       Recall (Sensitivity):      0.61     

       F1 Score:                  0.63

